Examining the LBM data more closely

In [2]:
from pprint import pprint
from dask_image.imread import imread
import napari
import dask_image
import dask
from pathlib import Path
import skimage.io
import tifffile
import scipy.io
import numpy as np

try:
    from icecream import ic, install, argumentToString
    install()
except ImportError:  # graceful fallback if icecream isn't installed.
    ic = lambda *a: None if not a else (a[0] if len(a) == 1 else a)  # noqa

install()

filename = "/v-data2/jeff_demas/Jeffs LBM paper data/Fig2"
raw = "/v-data2/jeff_demas/Jeffs LBM paper data/Fig2/20191122/mh89_hemisphere_FOV_50_550um_depth_250mW_dual_stimuli_30min_00001/data/mh89_hemisphere_FOV_50_550um_depth_250mW_dual_stimuli_30min_00001_00001.tif"

p = list(Path(filename).glob('**/*.tif'))
mats = list(Path(filename).glob('**/*.mat'))

Fig2_dataset_plane_n.mat, where n = 1:30, containing the following fields:

Y: motion-corrected single plane imaging data [nx,ny,nt]
pixelResolution: pixel sampling [um]
sizY: size of y [nx,ny,nt]
volumeRate: volume rate [Hz]

Processed data file:

Fig2_collated_caiman_output.mat, containing the following fields:

T_all: raw neuronal traces [K,nt]
nx: neuronal coordinate in the x direction [K,1], [um]
ny: neuronal coordinate in the y direction [K,1], [um]
nz: neuronal coordinate in the z direction [K,1], [um]


In [3]:
raw_tiffs = filename.rglob("**/*.tif")
caiman_output_planes = [x for x in mats if x.name in ['caiman_output_plane_1.mat', 'caiman_output_plane_16.mat','caiman_output_plane_30.mat']]
caiman_bg_output_planes = [x for x in mats if x.name in ['caiman_output_plane_1.mat', 'caiman_output_plane_16.mat','caiman_output_plane_30.mat']]
mh89_planes = [x for x in mats if x.name in ['Fig2_dataset_plane_1.mat', 'Fig2_dataset_plane_16.mat', 'Fig2_dataset_plane_30.mat']]
output = [x for x in mats if x.name == 'Fig2_collated_caiman_output.mat'][0]

AttributeError: 'str' object has no attribute 'rglob'

In [4]:
caiman_output_planes_dict = scipy.io.loadmat(caiman_output_planes[0])
arr_A_keep = caiman_output_planes_dict['A_keep']
arr_C_keep = caiman_output_planes_dict['C_keep']
arr_Cn = caiman_output_planes_dict['Cn']

NameError: name 'caiman_output_planes' is not defined

This is a nice way to view and retrieve ScanImage metadata.

In [ ]:
with open(p[0], 'rb') as fh:
    metadata = tifffile.read_scanimage_metadata(fh)
     #pprint(tifffile.read_scanimage_metadata(fh))

In [ ]:
r_tiff_file = tifffile.imread(p[0], aszarr=True)
r_arr = dask.array.from_zrr(tiff_file)

## Current Vaziri Implementation

In [5]:
with tifffile.TiffFile(p[0]) as tif:
    metadata = {}
    for tag in tif.pages[0].tags.values():
        tag_name, tag_value = tag.name, tag.value
        metadata[tag_name] = tag_value

In [4]:
metadims = {}
for k, v in metadata.items():
    if v in ['145', '144', 145, 144, 5104, '5104']:
        metadims[k] = v
        
pprint(metadims)

{'ImageLength': 144, 'ImageWidth': 145, 'RowsPerStrip': 144}


In [5]:
chans_order = np.array([ 1,  5,  6,  7,  8,  9,  2, 10, 11, 12, 13, 14, 15, 16, 17,
                            3, 18, 19, 20, 21, 22, 23,  4, 24, 25, 26, 27, 28, 29, 30]) - 1
n_planes = 30

In [6]:
tiff_file = tifffile.imread(p[0])
tiff_file = np.reshape(tiff_file, ( int(tiff_file.shape[0]/n_planes), n_planes,tiff_file.shape[1],tiff_file.shape[2]), order='A') # warnings are expected if the recording is split into many files or incomplete
tiff_file = np.swapaxes(tiff_file, 1, 3)
tiff_file = tiff_file[...,chans_order]

<tifffile.TiffFile 'pollen_calibrati…M_30x_00001.tif'> asarray failed to reshape (9030, 144, 145) to (6, 50, 30, 144, 145)


In [7]:
path_all_files = []
for i_dir in params["raw_data_dirs"]:
    tmp_paths = sorted(glob.glob(i_dir + "/**/*.tif", recursive=True))
    for this_tmp_path in tmp_paths:
        if (
                params["fname_must_contain"] in this_tmp_path and
                params["fname_must_NOT_contain"] not in this_tmp_path
        ):
            path_all_files.append(this_tmp_path)


NameError: name 'params' is not defined